## Generative Deep Learning

In [1]:
# auto reloading
%reload_ext autoreload
%autoreload 2

In [2]:
# import
import numpy as np
import keras
from myutils import *

/home/atul/anaconda/envs/dpl/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
text = open('../../datasets/nietzsche.txt').read().lower()

In [4]:
print("Corpus length: %s" % str(len(text)))

Corpus length: 600893


In [5]:
max_len = 60  # seq of characters to train
step = 3      # 
sents = []
next_char = []

In [6]:
for i in range(0, len(text)-max_len, step):
    sents.append(text[i: i+max_len])
    next_char.append(text[i+max_len])

In [7]:
sents[:3]

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not']

In [8]:
next_char[:3]

['r', 'n', ' ']

In [9]:
text[:180]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand wo'

In [10]:
st = "this is me this calling atul"

In [11]:
word_index = word_index_generator(text)
word_index

{'reduction': 0,
 'ranks;': 1,
 'true.': 2,
 'sensual,': 3,
 'self-destination,': 4,
 'experimentally': 5,
 'character"': 6,
 'human,': 7,
 'thus:': 8,
 'mystics,': 9,
 'immediately': 10,
 'solutions': 11,
 'lay': 12,
 'furious,': 13,
 'selfishness?': 14,
 'felt--one': 15,
 'superimposed': 16,
 'possessed,': 17,
 'gone': 18,
 'personality.': 19,
 'intercalated': 20,
 'well).': 21,
 'confidingness,': 22,
 'tire': 23,
 'will,"': 24,
 'pleasant,"': 25,
 'recollections': 26,
 'lyrists': 27,
 'devised,': 28,
 'es': 29,
 'obtained': 30,
 'mise': 31,
 'upward': 32,
 'shortness': 33,
 'vain': 34,
 'is,--that': 35,
 'on,': 36,
 '"morbidness"': 37,
 'obscurity,': 38,
 'false?': 39,
 'refinement,': 40,
 'nature--must': 41,
 'anxiety.': 42,
 'spirituality--anything': 43,
 'nature?': 44,
 'sedative': 45,
 'wrestler,': 46,
 'relatively': 47,
 'daubed': 48,
 'conquest': 49,
 'church).': 50,
 'choses': 51,
 'money--admiration.': 52,
 'away': 53,
 'aloud': 54,
 'impulse--the': 55,
 '"god"!': 56,
 'barb

In [12]:
char_index = char_index_generator(text)
char_index

{'\n': 36,
 ' ': 47,
 '!': 55,
 '"': 39,
 "'": 24,
 '(': 34,
 ')': 22,
 ',': 25,
 '-': 49,
 '.': 27,
 '0': 1,
 '1': 46,
 '2': 11,
 '3': 32,
 '4': 14,
 '5': 18,
 '6': 15,
 '7': 41,
 '8': 31,
 '9': 42,
 ':': 4,
 ';': 9,
 '=': 7,
 '?': 53,
 '[': 26,
 ']': 5,
 '_': 40,
 'a': 38,
 'b': 21,
 'c': 8,
 'd': 52,
 'e': 10,
 'f': 23,
 'g': 2,
 'h': 28,
 'i': 29,
 'j': 56,
 'k': 30,
 'l': 44,
 'm': 35,
 'n': 54,
 'o': 50,
 'p': 16,
 'q': 20,
 'r': 12,
 's': 37,
 't': 17,
 'u': 51,
 'v': 13,
 'w': 6,
 'x': 3,
 'y': 0,
 'z': 48,
 'ä': 45,
 'æ': 33,
 'é': 19,
 'ë': 43}

### Vectorizing the sentences for the characters

In [18]:
x = np.zeros((len(sents), max_len, len(char_index)), dtype=np.bool)
y = np.zeros((len(sents), len(char_index)), dtype=np.bool)

for idx,sent in enumerate(sents):  
    for cdx, char in enumerate(sent):
        x[idx, cdx, char_index[char]] = 1
        y[idx, char_index[next_char[idx]]] = 1
    

In [19]:
x[:2]

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])

In [20]:
y[:2]

array([[False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True, False, False]])

### Building the network

In [21]:
from keras import layers

In [23]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(max_len, len(char_index))))
model.add(layers.Dense(len(char_index), activation='softmax'))

In [24]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.01), loss='categorical_crossentropy')